## Heston-Implied Volatility Surfaces Télécharge les options via `yfinance`, calcule les prix/IV selon Heston, puis trace les surfaces calls/puts. 

In [13]:
import math from dataclasses import dataclass from typing import Dict, List, Tuple  import numpy as np import pandas as pd import plotly.graph_objects as go import yfinance as yf  MAX_LOOKAHEAD_YEARS = 3 MIN_MATURITY = 0.1 PHI_MAX = 200.0 PHI_STEPS = 2000 

In [14]:
def fetch_spot(symbol: str) -> float:     ticker = yf.Ticker(symbol)     hist = ticker.history(period="1d")     if hist.empty:         raise RuntimeError("Unable to retrieve spot price.")     return float(hist['Close'].iloc[-1])   def select_monthly_expirations(expirations: List[str]) -> List[str]:     today = pd.Timestamp.utcnow().date()     limit_date = today + pd.Timedelta(days=365 * MAX_LOOKAHEAD_YEARS)     monthly: Dict[Tuple[int, int], Tuple[pd.Timestamp, str]] = {}     for exp in expirations:         exp_ts = pd.Timestamp(exp)         exp_date = exp_ts.date()         if not (today < exp_date <= limit_date):             continue         key = (exp_date.year, exp_date.month)         if key not in monthly or exp_ts < monthly[key][0]:             monthly[key] = (exp_ts, exp)     return [item[1] for item in sorted(monthly.values(), key=lambda x: x[0])]   def download_options(symbol: str, option_type: str) -> pd.DataFrame:     ticker = yf.Ticker(symbol)     spot = fetch_spot(symbol)     expirations = ticker.options     selected = select_monthly_expirations(expirations)     rows = []     now = pd.Timestamp.utcnow().tz_localize(None)     for expiry in selected:         expiry_dt = pd.Timestamp(expiry)         T = max((expiry_dt - now).total_seconds() / (365.0 * 24 * 3600), 0.0)         data = ticker.option_chain(expiry).calls if option_type == 'call' else ticker.option_chain(expiry).puts         price_col = 'C_mkt' if option_type == 'call' else 'P_mkt'         for _, row in data.iterrows():             rows.append({'S0': spot, 'K': float(row['strike']), 'T': T, price_col: float(row['lastPrice'])})     return pd.DataFrame(rows) 

In [15]:
def bs_price(S0, K, T, vol, r, option_type):     if T <= 0 or vol <= 0:         intrinsic = max(0.0, S0 - K * math.exp(-r * T))         return intrinsic if option_type == 'call' else intrinsic - S0 + K * math.exp(-r * T)     sqrt_T = math.sqrt(T)     v = vol * sqrt_T     d1 = (math.log(S0 / K) + (r + 0.5 * vol * vol) * T) / v     d2 = d1 - v     nd1 = 0.5 * (1.0 + math.erf(d1 / math.sqrt(2.0)))     nd2 = 0.5 * (1.0 + math.erf(d2 / math.sqrt(2.0)))     call = S0 * nd1 - K * math.exp(-r * T) * nd2     if option_type == 'call':         return call     return call - S0 + K * math.exp(-r * T) def implied_vol(price, S0, K, T, r, option_type):     intrinsic = bs_price(S0, K, T, 0.0, r, option_type)     if price <= max(intrinsic, 1e-12):         return 0.0     low, high = 1e-6, 1.0     high_price = bs_price(S0, K, T, high, r, option_type)     while high_price < price and high < 5.0:         high *= 2.0         high_price = bs_price(S0, K, T, high, r, option_type)     if high_price < price:         return float('nan')     for _ in range(100):         mid = 0.5 * (low + high)         mid_price = bs_price(S0, K, T, mid, r, option_type)         if abs(mid_price - price) < 1e-6:             return mid         if mid_price > price:             high = mid         else:             low = mid     return 0.5 * (low + high) 

In [16]:
def heston_probability(S0, K, T, r, kappa, theta, sigma, rho, v0, Pnum):     n = PHI_STEPS     if n % 2 == 0:         n += 1     phi = np.linspace(1e-5, PHI_MAX, n)     u = 0.5 if Pnum == 1 else -0.5     b = kappa - rho * sigma if Pnum == 1 else kappa     a_param = kappa * theta     x = math.log(S0)     d = np.sqrt((rho * sigma * 1j * phi - b) ** 2 - sigma ** 2 * (2 * u * 1j * phi - phi ** 2))     g = (b - rho * sigma * 1j * phi + d) / (b - rho * sigma * 1j * phi - d)     exp_dt = np.exp(-d * T)     log_term = np.log((1.0 - g * exp_dt) / (1.0 - g))     C = r * 1j * phi * T + (a_param / (sigma ** 2)) * ((b - rho * sigma * 1j * phi + d) * T - 2.0 * log_term)     D = ((b - rho * sigma * 1j * phi + d) / (sigma ** 2)) * ((1.0 - exp_dt) / (1.0 - g * exp_dt))     integrand = np.real(np.exp(C + D * v0 + 1j * phi * (x - math.log(K))) / (1j * phi))     h = phi[1] - phi[0]     integral = h / 3.0 * (integrand[0] + integrand[-1] + 4.0 * np.sum(integrand[1:-1:2]) + 2.0 * np.sum(integrand[2:-2:2]))     return 0.5 + (1.0 / math.pi) * integral def heston_call_price(S0, K, T, r, params):     P1 = heston_probability(S0, K, T, r, **params, Pnum=1)     P2 = heston_probability(S0, K, T, r, **params, Pnum=2)     return S0 * P1 - K * math.exp(-r * T) * P2 def heston_put_price(S0, K, T, r, params):     call = heston_call_price(S0, K, T, r, params)     return call - S0 + K * math.exp(-r * T) 

In [17]:
def add_heston_iv(df, option_type, params, r):     df_out = df.copy()     ivs = []     prices = []     for row in df.itertuples(index=False):         S0, K, T = float(row.S0), float(row.K), float(row.T)         if option_type == 'call':             price = heston_call_price(S0, K, T, r, params)         else:             price = heston_put_price(S0, K, T, r, params)         iv = implied_vol(price, S0, K, T, r, option_type)         ivs.append(iv)         prices.append(price)     df_out['heston_price'] = prices     df_out['iv_heston'] = ivs     return df_out def prepare_surface(df, strike_width):     spot = float(df['S0'].median())     lower = math.ceil((spot - strike_width) / 10.0) * 10.0     upper = math.ceil((spot + strike_width) / 10.0) * 10.0     filtered = df[(df['K'] >= lower) & (df['K'] <= upper) & (df['T'] >= MIN_MATURITY)].copy()     filtered = filtered.sort_values(['T', 'K']).reset_index(drop=True)     surface = filtered.pivot_table(index='T', columns='K', values='iv_heston', aggfunc='mean')     surface = surface.sort_index().sort_index(axis=1).interpolate(axis=1, limit_direction='both').loc[MIN_MATURITY:]     return filtered, surface, spot def plot_surface(surface, spot, title):     k_vals = surface.columns.to_numpy(dtype=float)     t_vals = surface.index.to_numpy(dtype=float)     KK, TT = np.meshgrid(k_vals, t_vals)     data = surface.to_numpy(dtype=float)     mean = float(np.nanmean(data))     std = float(np.nanstd(data))     fig = go.Figure(data=[go.Surface(x=KK, y=TT, z=data, colorscale='Viridis', showscale=True)])     fig.update_layout(title=title, scene=dict(xaxis_title=f'Strike K — spot ≈ {spot:.2f}', yaxis_title='T (years)', zaxis=dict(title='IV', range=[mean - 2*std, mean + 2*std])))     return fig 

In [18]:
symbol = 'SPY' strike_window = 100.0 risk_free_rate = 0.02 heston_params = {'kappa': 2.0, 'theta': 0.04, 'sigma': 0.5, 'rho': -0.7, 'v0': 0.04}  calls_df = download_options(symbol, 'call') puts_df = download_options(symbol, 'put') calls_heston = add_heston_iv(calls_df, 'call', heston_params, risk_free_rate) puts_heston = add_heston_iv(puts_df, 'put', heston_params, risk_free_rate) calls_data, calls_surface, spot = prepare_surface(calls_heston, strike_window) puts_data, puts_surface, _ = prepare_surface(puts_heston, strike_window) fig_calls = plot_surface(calls_surface, spot, 'Call IV Surface (Heston)') fig_puts = plot_surface(puts_surface, spot, 'Put IV Surface (Heston)') fig_calls.show() fig_puts.show() 